# 18. 번역기를 만들어보자

In [1]:
import os
import pandas as pd
import numpy as np
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 208906


,eng,fra,cc
110230,That isn't really my problem.,Ce n'est pas vraiment mon problème.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
149085,I would never question his honesty.,Je ne douterais jamais de son honnêteté.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
5552,Is this love?,Est-ce de l'amour ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
60724,It's pretty incredible.,C'est plutôt incroyable.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
121412,Have you ever opened this door?,Avez-vous déjà ouvert cette porte ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
17248,We're separated.,Nous sommes séparés.
7330,I canceled it.,Je l'ai annulée.
3763,"OK, you win.","D'accord, tu as gagné."
5315,I'm a priest.,Je suis prêtre.
4358,You are big.,Vous êtes grands.


## 프로그레싱

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

In [5]:
def preprocess_sentence_decoder(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    sentence = sentence.split(" ")
    
    return sentence

In [6]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence_decoder(x))

In [7]:
lines.eng.sample(5)

28290    [what, do, we, do, now, ?]
29780     [everybody, is, alive, .]
17888       [are, you, blushing, ?]
17520        [you, heard, wrong, .]
24601    [i, gave, him, a, call, .]
Name: eng, dtype: object

In [8]:
lines.fra.sample(5)

18962    [<start>, je, me, suis, cogn, le, genou, ., <e...
26310          [<start>, non, ne, le, fais, pas, ., <end>]
31381        [<start>, je, veux, que, tu, aides, ., <end>]
16937                  [<start>, tom, est, ravi, ., <end>]
15869       [<start>, maintenant, va, t, amuser, ., <end>]
Name: fra, dtype: object

In [9]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)

In [10]:
print(eng_tokenizer.word_index)

{'.': 1, 'i': 2, 'you': 3, '?': 4, 'it': 5, 'tom': 6, 's': 7, 'is': 8, 'a': 9, 're': 10, 'm': 11, 'we': 12, 't': 13, 'that': 14, 'he': 15, 'the': 16, 'me': 17, 'was': 18, 'this': 19, 'to': 20, 'are': 21, '!': 22, 'do': 23, 'can': 24, 'they': 25, 'my': 26, 'go': 27, 'your': 28, 'have': 29, 'don': 30, 'not': 31, 'll': 32, 'be': 33, 'she': 34, 'no': 35, 'let': 36, 'what': 37, 'did': 38, 'here': 39, 'like': 40, 'all': 41, 'in': 42, 'get': 43, 'how': 44, 'up': 45, 'on': 46, 'very': 47, 'need': 48, 'one': 49, 'want': 50, 'him': 51, 'love': 52, 'out': 53, 'please': 54, 'who': 55, 'come': 56, 'know': 57, 'just': 58, 'take': 59, 'stop': 60, 'now': 61, 'so': 62, 'us': 63, 'look': 64, 'there': 65, 'got': 66, 'help': 67, 'of': 68, 'too': 69, 'for': 70, 'good': 71, 'see': 72, 'at': 73, 've': 74, 'will': 75, 'keep': 76, 'am': 77, 'had': 78, 'has': 79, 'were': 80, 'back': 81, 'give': 82, 'stay': 83, 'home': 84, 'saw': 85, 'feel': 86, 'must': 87, 'alone': 88, 'happy': 89, 'where': 90, 'try': 91, 'leav

In [11]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:3]

[[27, 1], [27, 1], [27, 1]]

In [12]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)

In [13]:
target_text[:3]

[[1, 74, 9, 2], [1, 350, 3, 2], [1, 27, 516, 9, 2]]

In [14]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4619
프랑스어 단어장의 크기 : 7388


In [15]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [16]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4619
프랑스어 단어장의 크기 : 7388
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [17]:
sos_token = '<start>'
eos_token = '<end>'

encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [18]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [19]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [20]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


## 학습

In [21]:
encoder_inputs = Input(shape=(None,))
# encoder embedding
enc_emb = Embedding(eng_vocab_size, 256, input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [22]:
decoder_inputs = Input(shape=(None,))
# decoder embedding
dec_emb = Embedding(fra_vocab_size, 256)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1182464     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1891328     input_2[0][0]                    
______________________________________________________________________________________________

In [24]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [25]:
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
938/938 [==============================] - 47s 20ms/step - loss: 1.4630 - acc: 0.7718 - val_loss: 1.2080 - val_acc: 0.8087
Epoch 2/50
938/938 [==============================] - 16s 17ms/step - loss: 1.1061 - acc: 0.8237 - val_loss: 1.0543 - val_acc: 0.8323
Epoch 3/50
938/938 [==============================] - 16s 17ms/step - loss: 0.9778 - acc: 0.8415 - val_loss: 0.9641 - val_acc: 0.8446
Epoch 4/50
938/938 [==============================] - 16s 17ms/step - loss: 0.8918 - acc: 0.8531 - val_loss: 0.9129 - val_acc: 0.8524
Epoch 5/50
938/938 [==============================] - 16s 17ms/step - loss: 0.8302 - acc: 0.8617 - val_loss: 0.8672 - val_acc: 0.8587
Epoch 6/50
938/938 [==============================] - 16s 17ms/step - loss: 0.7808 - acc: 0.8695 - val_loss: 0.8364 - val_acc: 0.8635
Epoch 7/50
938/938 [==============================] - 16s 17ms/step - loss: 0.7443 - acc: 0.8767 - val_loss: 0.8259 - val_acc: 0.8673
Epoch 8/50
938/938 [==============================] - 16s 17ms

## 평가

In [26]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1182464   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,707,776
Trainable params: 1,707,776
Non-trainable params: 0
_________________________________________________________________


In [27]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [28]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    1891328     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [29]:
ng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [31]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [32]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [33]:
for seq_index in [1,100,301,777,2222]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: we ll manage . 
정답 문장: nous y arriverons . 
번역기가 번역한 문장:  nous . . . ? ? ? 
-----------------------------------
입력 문장: i had to see you . 
정답 문장: il m a fallu vous voir . 
번역기가 번역한 문장:  il me me que . . 
-----------------------------------
입력 문장: his son is sick . 
정답 문장: son fils est malade . 
번역기가 번역한 문장:  aime votre la la 
-----------------------------------
입력 문장: he has a picasso . 
정답 문장: il a un picasso . 
번역기가 번역한 문장:  il d . . . ! . . 
-----------------------------------
입력 문장: i m a law student . 
정답 문장: je suis tudiante en droit . 
번역기가 번역한 문장:  je un la la la l


# 회고

### 전문적인 번역기와 비교하면 장난수준이지만 그래도 번역을 이렇게나마 해볼 수 있어서 즐거운 프로젝트였다.
### 아직 미숙한 점은 여러 방식과 모델학습을 통해 보완해가면 더 나은 결과물을 낼 수 있을 것 같다.
### 여러모로 흥미롭고 실전에서 사용해볼 수 있는 프로젝트여서 나름 뿌듯했다.
### 영어 to 프랑스어 라서 조금 아쉬웠고, 한국어나 스페인어를 한번 더 해보고 싶다는 생각을 했다.
### 요즘 거의 99%에 가까운 Deepl이라는 번역기 나와서 서로 비교해보면서 보완을 해보면 좋겠다.
### 인풋과 아웃풋,그리고 과정에 대해 한번 더 배울수 있는 시간이었다.
